<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Mw-Pradeep/bioco_mw_veiledet_regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.2 MB/s eta 0:00:00


Importering av nødvendig bibliotek og pakker

In [ ]:
import pandas as pd
import numpy as np
import copy
import math
from statistics import mean, stdev

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline

In [ ]:
# Modeller
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [ ]:
# Hyperoptimalisering
import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
# optuna.logging.set_verbosity(optuna.logging.WARNING)

# Egendefinerte moduler

In [ ]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_trening_validering_testing_evaluering.py /content/moduler_trening_validering_testing_evaluering.py

/content


In [ ]:
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_enzym
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_dag
from moduler_trening_validering_testing_evaluering import trening_validering_oppdeling_dag

from moduler_trening_validering_testing_evaluering import enzymfordeling
from moduler_trening_validering_testing_evaluering import n_splitt_enzym
from moduler_trening_validering_testing_evaluering import Resultat_evaluering
from moduler_trening_validering_testing_evaluering import res_eval_opt
from moduler_trening_validering_testing_evaluering import res_eval
from moduler_trening_validering_testing_evaluering import Resultat_dicts
from moduler_trening_validering_testing_evaluering import evaluering_beregning
from moduler_trening_validering_testing_evaluering import enzymtype_rawmaterial_prediksjon

In [ ]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_optimalisering_evaluering_veiledet.py /content/moduler_optimalisering_evaluering_veiledet.py

/content


In [ ]:
from moduler_optimalisering_evaluering_veiledet import trening_rfr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_knr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_svr_enzym

from moduler_optimalisering_evaluering_veiledet import trening_rfr_dag
from moduler_optimalisering_evaluering_veiledet import trening_knr_dag
from moduler_optimalisering_evaluering_veiledet import trening_svr_dag

from moduler_optimalisering_evaluering_veiledet import veiledet_modell_evaluering_1
from moduler_optimalisering_evaluering_veiledet import veiledet_modell_evaluering_2

# Importering av relevant data

In [ ]:
# Velger første kolonne med dato og tid som index
data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Design produksjon
data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data_design.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
data.index = pd.to_datetime(data.index,
                            format='%Y-%m-%d %H:%M:%S')

data_design.index = pd.to_datetime(data_design.index,
                                   format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [ ]:
#Tilfeldighetsfrø
random_seed = 2024

# Lagring av resultater

In [ ]:
# Oppretter instans for lagring av resultater
res = Resultat_dicts()

# Setter fast testandel på 20% og definerer treningsandeler

In [ ]:
# Fast testsandel
test_andel = 0.2

# Treningsandeler til undersøkelse
treningsandeler = [0.25, 0.5, 0.75]

# Parametere til hyperoptimalisering

In [ ]:
# Antall runder til optimalisering

# Antall runder til hel treningsandel
n_runder_hel = 100

# Antall runder til mindre treningsandeler
n_runder_andel = 100

# Parametervalg for kryssvalidering
n_gjentagelser = 2

# NIR målinger inkludert

## Hele datasett

## Alternativ 1: Hele treningssett

In [ ]:
# Setter treningsandel
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data,
                                   test_andel=test_andel)

In [ ]:
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

rskf = RepeatedStratifiedKFold(n_splits=n_splitt,
                                 n_repeats=n_gjentagelser,
                                 random_state=random_seed)

statistisk_res = Resultat_evaluering()
# Går igjennom kryssvalidering
for train, val in rskf.split(trening_markert,
                              enzymtyper_markert):
  # Markerte folds
  trenings_fold_markert = trening_markert.iloc[train, :]
  val_fold = trening_markert.iloc[val, :]

  # Markert respons
  y_trening_fold = trenings_fold_markert.iloc[:, -1]
  y_val_fold = val_fold.iloc[:, -1]


  y_trening_pred, y_trening_fold = enzymtype_rawmaterial_prediksjon(rå_data, trenings_fold_markert, trenings_fold_markert, 'Mw', 'mean')
  y_val_pred, y_val_fold = enzymtype_rawmaterial_prediksjon(rå_data, trenings_fold_markert, val_fold, 'Mw', 'mean')
  y_test_pred, y_test = enzymtype_rawmaterial_prediksjon(rå_data, trenings_fold_markert, testsett, 'Mw', 'mean')


  statistisk_res.evaluering(y_trening_fold,
                            y_trening_pred,
                            y_val_fold,
                            y_val_pred,
                            y_test,
                            y_test_pred)

In [ ]:
statistisk_res.beskriv_statistikk()

RMSE (trening): Mean = 808.3381, Std = 15.5707
RMSE (val): Mean = 819.0448, Std = 142.1240
RMSE (test): Mean = 785.3374, Std = 2.0937
R2 (trening): Mean = 0.8852, Std = 0.0049
R2 (val): Mean = 0.8753, Std = 0.0443
R2 (test): Mean = 0.8908, Std = 0.0006
MAE (trening): Mean = 550.8456, Std = 10.5696
MAE (val): Mean = 567.7787, Std = 85.1418
MAE (test): Mean = 567.4851, Std = 4.0240
MAPE (trening): Mean = 0.0844, Std = 0.0016
MAPE (val): Mean = 0.0868, Std = 0.0138
MAPE (test): Mean = 0.0863, Std = 0.0006


In [ ]:
# Finner antall splitt etter minste antall av enzymtype
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_hel_opt,
                             dict_res=res.dict_res_1_hel,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder_hel,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

res_1_hel = pd.DataFrame(res.dict_res_1_hel)
res_1_hel_opt = pd.DataFrame(res.dict_res_1_hel_opt)

res_1_hel.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_hel.csv')
res_1_hel_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_hel_opt.csv')

[I 2024-06-16 22:35:04,150] A new study created in memory with name: no-name-d980f7ac-c40d-4934-8e04-9896bd56a9c3
[I 2024-06-16 22:35:04,545] Trial 1 finished with value: 531.1001133053591 and parameters: {'n_neighbors': 10, 'weights': 'distance', 'p': 1}. Best is trial 1 with value: 531.1001133053591.
[I 2024-06-16 22:35:04,583] Trial 0 finished with value: 618.0778585242214 and parameters: {'n_neighbors': 1, 'weights': 'uniform', 'p': 7}. Best is trial 1 with value: 531.1001133053591.
[I 2024-06-16 22:35:04,944] Trial 3 finished with value: 558.6516586330999 and parameters: {'n_neighbors': 1, 'weights': 'distance', 'p': 1}. Best is trial 1 with value: 531.1001133053591.
[I 2024-06-16 22:35:05,013] Trial 2 finished with value: 571.0650654915044 and parameters: {'n_neighbors': 8, 'weights': 'uniform', 'p': 4}. Best is trial 1 with value: 531.1001133053591.
[I 2024-06-16 22:35:05,250] Trial 5 pruned. 
[I 2024-06-16 22:35:05,361] Trial 4 finished with value: 555.3284715863729 and paramet

KeyboardInterrupt: 

Lagring av resultater

In [ ]:
res_1_hel = pd.DataFrame(res.dict_res_1_hel)
res_1_hel_opt = pd.DataFrame(res.dict_res_1_hel_opt)

res_1_hel.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_hel.csv')
res_1_hel_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_hel_opt.csv')

Velger beste modell

## Alternativ 1: Forskjellige treningsandeler

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, _, _, _ = trening_testsett_oppdeling_enzym(rå_data,
                                                       trening,
                                                       test_andel=ubenyttet_andel,
                                                       random_seed=random_seed)
  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert

  # Finner antall splitt etter minste antall av enzymtype
  n_splitt = n_splitt_enzym(rå_data,
                            treningssett_markert)

  # Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
  veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_hel_opt,
                               dict_res=res.dict_res_1_hel,
                               treningssett_markert=treningssett_markert,
                               enzymtyper_treningssett_markert=enzymtyper_treningssett_markert,
                               testsett=testsett,
                               n_splitt=n_splitt,
                               n_gjentagelser=n_gjentagelser,
                               n_runder=n_runder_andel,
                               test_andel=test_andel,
                               trenings_andel=trenings_andel,
                               random_seed=random_seed)

  res_1_hel = pd.DataFrame(res.dict_res_1_hel)
  res_1_hel_opt = pd.DataFrame(res.dict_res_1_hel_opt)

  res_1_hel.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_hel.csv')
  res_1_hel_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_hel_opt.csv')

[I 2024-05-17 23:18:14,045] A new study created in memory with name: no-name-91ac8442-ba40-4ffb-b441-c2d3cdc95bd9
[I 2024-05-17 23:18:14,226] Trial 0 finished with value: 979.8598764467637 and parameters: {'n_neighbors': 10, 'weights': 'uniform', 'p': 3}. Best is trial 0 with value: 979.8598764467637.
[I 2024-05-17 23:18:14,251] Trial 1 finished with value: 798.4398390770775 and parameters: {'n_neighbors': 7, 'weights': 'distance', 'p': 10}. Best is trial 1 with value: 798.4398390770775.
[I 2024-05-17 23:18:14,432] Trial 3 finished with value: 766.7070066977114 and parameters: {'n_neighbors': 3, 'weights': 'distance', 'p': 9}. Best is trial 3 with value: 766.7070066977114.
[I 2024-05-17 23:18:14,436] Trial 2 finished with value: 683.0462603995694 and parameters: {'n_neighbors': 2, 'weights': 'uniform', 'p': 4}. Best is trial 2 with value: 683.0462603995694.
[I 2024-05-17 23:18:14,602] Trial 5 finished with value: 795.234479981623 and parameters: {'n_neighbors': 8, 'weights': 'distance'

Lagring av resultater

In [ ]:
res_1_hel = pd.DataFrame(res.dict_res_1_hel)
res_1_hel_opt = pd.DataFrame(res.dict_res_1_hel_opt)

res_1_hel.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_hel.csv')
res_1_hel_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_res_1_hel_opt.csv')

## Alternativ 2: Hele treningssett

In [ ]:
# Definerer treningsandel
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

Enzymtyper Fordeling: Test
            Frekvens  Andel
EnzymeCode                 
A2                17  22.08
B                 15  19.48
C                 14  18.18
E                 14  18.18
D                 12  15.58
A1                 5   6.49


In [ ]:
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)


statistisk_res = Resultat_evaluering()
# Går igjennom kryssvalidering
trenings_sett, val_sett\
= trening_validering_oppdeling_dag(rå_data,
                                    trening_markert,
                                    n_splitt=n_splitt,
                                    test_ekskluder_kalkun_kylling=True)

for i, j in zip(trenings_sett, val_sett):
    train = trenings_sett[i]
    val = val_sett[j]

    # Markerte folds
    trenings_fold_markert = trening_markert.loc[train, :]
    val_fold = trening_markert.loc[val, :]

    # Markert respons
    y_trening_fold = trenings_fold_markert.iloc[:, -1]
    y_val_fold = val_fold.iloc[:, -1]


    y_trening_pred, y_trening_fold = enzymtype_rawmaterial_prediksjon(rå_data, trenings_fold_markert, trenings_fold_markert, 'Mw', 'mean')
    y_val_pred, y_val_fold = enzymtype_rawmaterial_prediksjon(rå_data, trenings_fold_markert, val_fold, 'Mw', 'mean')
    y_test_pred, y_test = enzymtype_rawmaterial_prediksjon(rå_data, trenings_fold_markert, testsett, 'Mw', 'mean')


    statistisk_res.evaluering(y_trening_fold,
                              y_trening_pred,
                              y_val_fold,
                              y_val_pred,
                              y_test,
                              y_test_pred)

In [ ]:
statistisk_res.beskriv_statistikk()

RMSE (trening): Mean = 788.6836, Std = 18.3586
RMSE (val): Mean = 916.3121, Std = 184.4370
RMSE (test): Mean = 912.7168, Std = 49.2854
R2 (trening): Mean = 0.8943, Std = 0.0051
R2 (val): Mean = 0.7959, Std = 0.0784
R2 (test): Mean = 0.8157, Std = 0.0197
MAE (trening): Mean = 550.1109, Std = 14.6742
MAE (val): Mean = 686.4432, Std = 145.1057
MAE (test): Mean = 640.4107, Std = 22.9140
MAPE (trening): Mean = 0.0847, Std = 0.0019
MAPE (val): Mean = 0.1081, Std = 0.0205
MAPE (test): Mean = 0.0816, Std = 0.0027


In [ ]:
# Finner antall splitt etter minste antall av enzymtype
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_hel_opt,
                             dict_res=res.dict_res_2_hel,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder_hel,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

res_2_hel = pd.DataFrame(res.dict_res_2_hel)
res_2_hel_opt = pd.DataFrame(res.dict_res_2_hel_opt)

res_2_hel.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_hel.csv')
res_2_hel_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_hel_opt.csv')

[I 2024-05-18 00:35:56,364] A new study created in memory with name: no-name-079ad98a-3bdd-40b4-9c25-641d96eda6c8
[I 2024-05-18 00:36:09,256] Trial 1 finished with value: 659.2270040828968 and parameters: {'n_neighbors': 3, 'weights': 'distance', 'p': 10}. Best is trial 1 with value: 659.2270040828968.
[I 2024-05-18 00:36:09,494] Trial 0 finished with value: 647.919730427664 and parameters: {'n_neighbors': 5, 'weights': 'uniform', 'p': 8}. Best is trial 0 with value: 647.919730427664.
[I 2024-05-18 00:36:22,562] Trial 3 finished with value: 648.5580102343852 and parameters: {'n_neighbors': 9, 'weights': 'uniform', 'p': 3}. Best is trial 0 with value: 647.919730427664.
[I 2024-05-18 00:36:22,570] Trial 2 finished with value: 625.8855558039942 and parameters: {'n_neighbors': 6, 'weights': 'distance', 'p': 8}. Best is trial 2 with value: 625.8855558039942.
[I 2024-05-18 00:36:35,294] Trial 4 finished with value: 651.6054854083256 and parameters: {'n_neighbors': 3, 'weights': 'distance', '

Lagring av resultater

In [ ]:
res_2_hel = pd.DataFrame(res.dict_res_2_hel)
res_2_hel_opt = pd.DataFrame(res.dict_res_2_hel_opt)

res_2_hel.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_hel.csv')
res_2_hel_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_hel_opt.csv')

NameError: name 'pd' is not defined

Velger beste modell

## Alternativ 2: Forskjellige treningssandeler

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, _, _\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       test_ekskluder_kalkun_kylling=True)

  else:
    treningssett_markert = trening_markert

  #Enzymfordeling
  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)
  print(enzym_treningssett_markert)

  # n_splitt
  n_splitt = n_splitt_enzym(rå_data, treningssett_markert)

  veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_hel_opt,
                               dict_res=res.dict_res_2_hel,
                               rå_data=rå_data,
                               treningssett_markert=treningssett_markert,
                               testsett=testsett,
                               n_splitt=n_splitt,
                               n_runder=n_runder_andel,
                               test_andel=test_andel,
                               trenings_andel=trenings_andel,
                               random_seed=random_seed)

  res_2_hel = pd.DataFrame(res.dict_res_2_hel)
  res_2_hel_opt = pd.DataFrame(res.dict_res_2_hel_opt)

  res_2_hel.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_hel.csv')
  res_2_hel_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_hel_opt.csv')

[I 2024-05-18 01:30:11,036] A new study created in memory with name: no-name-ab324c55-234a-4a3b-a632-06a99f1dc6ac


Enzymtyper Fordeling: Treningssett markert andel: 0.25
            Frekvens  Andel
EnzymeCode                 
A2                50  29.59
C                 31  18.34
B                 28  16.57
D                 28  16.57
E                 28  16.57
A1                 4   2.37


[I 2024-05-18 01:30:18,265] Trial 1 finished with value: 1053.2247590798306 and parameters: {'n_neighbors': 10, 'weights': 'uniform', 'p': 9}. Best is trial 1 with value: 1053.2247590798306.
[I 2024-05-18 01:30:18,325] Trial 0 finished with value: 1051.649577302112 and parameters: {'n_neighbors': 9, 'weights': 'uniform', 'p': 10}. Best is trial 0 with value: 1051.649577302112.
[I 2024-05-18 01:30:25,779] Trial 2 finished with value: 862.4100501564127 and parameters: {'n_neighbors': 9, 'weights': 'uniform', 'p': 1}. Best is trial 2 with value: 862.4100501564127.
[I 2024-05-18 01:30:26,098] Trial 3 finished with value: 977.3587008540369 and parameters: {'n_neighbors': 6, 'weights': 'uniform', 'p': 5}. Best is trial 2 with value: 862.4100501564127.
[I 2024-05-18 01:30:32,881] Trial 4 finished with value: 1047.091422274682 and parameters: {'n_neighbors': 1, 'weights': 'distance', 'p': 5}. Best is trial 2 with value: 862.4100501564127.
[I 2024-05-18 01:30:33,195] Trial 5 finished with value

Enzymtyper Fordeling: Treningssett markert andel: 0.5
            Frekvens  Andel
EnzymeCode                 
A2                75  36.41
C                 33  16.02
E                 31  15.05
B                 30  14.56
D                 29  14.08
A1                 8   3.88


[I 2024-05-18 01:59:15,718] Trial 0 finished with value: 715.9995736976133 and parameters: {'n_neighbors': 4, 'weights': 'distance', 'p': 7}. Best is trial 0 with value: 715.9995736976133.
[I 2024-05-18 01:59:15,992] Trial 1 finished with value: 742.1388700783657 and parameters: {'n_neighbors': 2, 'weights': 'uniform', 'p': 2}. Best is trial 0 with value: 715.9995736976133.
[I 2024-05-18 01:59:27,500] Trial 2 finished with value: 711.1360624998763 and parameters: {'n_neighbors': 2, 'weights': 'distance', 'p': 1}. Best is trial 2 with value: 711.1360624998763.
[I 2024-05-18 01:59:28,053] Trial 3 finished with value: 709.5802296284564 and parameters: {'n_neighbors': 6, 'weights': 'distance', 'p': 7}. Best is trial 3 with value: 709.5802296284564.
[I 2024-05-18 01:59:38,607] Trial 4 finished with value: 815.0545851864974 and parameters: {'n_neighbors': 1, 'weights': 'uniform', 'p': 7}. Best is trial 3 with value: 709.5802296284564.
[I 2024-05-18 01:59:40,205] Trial 5 finished with value: 

Enzymtyper Fordeling: Treningssett markert andel: 0.75
            Frekvens  Andel
EnzymeCode                 
A2               108  38.30
E                 46  16.31
C                 41  14.54
B                 40  14.18
D                 35  12.41
A1                12   4.26


[I 2024-05-18 02:45:49,379] Trial 1 finished with value: 709.1725033646064 and parameters: {'n_neighbors': 2, 'weights': 'uniform', 'p': 7}. Best is trial 1 with value: 709.1725033646064.
[I 2024-05-18 02:45:49,506] Trial 0 finished with value: 692.2895213693223 and parameters: {'n_neighbors': 7, 'weights': 'uniform', 'p': 10}. Best is trial 0 with value: 692.2895213693223.
[I 2024-05-18 02:46:03,022] Trial 3 finished with value: 672.2134016841521 and parameters: {'n_neighbors': 5, 'weights': 'uniform', 'p': 3}. Best is trial 3 with value: 672.2134016841521.
[I 2024-05-18 02:46:03,199] Trial 2 finished with value: 709.3142258514174 and parameters: {'n_neighbors': 8, 'weights': 'uniform', 'p': 1}. Best is trial 3 with value: 672.2134016841521.
[I 2024-05-18 02:46:16,483] Trial 4 finished with value: 698.9720638734316 and parameters: {'n_neighbors': 2, 'weights': 'uniform', 'p': 10}. Best is trial 3 with value: 672.2134016841521.
[I 2024-05-18 02:46:16,589] Trial 5 finished with value: 6

Lagring av resultater

In [ ]:
res_2_hel = pd.DataFrame(res.dict_res_2_hel)
res_2_hel_opt = pd.DataFrame(res.dict_res_2_hel_opt)

res_2_hel.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_hel.csv')
res_2_hel_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_hel_opt.csv')

## Designproduksjon

## Alternativ 1: Hele treningssett

In [ ]:
# Definere treningsandel
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design,
                                   test_andel=test_andel)

In [ ]:
# Finner antall splitt etter minste antall av enzymtype
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_design_opt,
                             dict_res=res.dict_res_1_design,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder_hel,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

res_1_design = pd.DataFrame(res.dict_res_1_design)
res_1_design_opt = pd.DataFrame(res.dict_res_1_design_opt)

res_1_design.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_1_design.csv')
res_1_design_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_1_design_opt.csv')

[I 2024-05-18 03:32:28,088] A new study created in memory with name: no-name-80d52d41-a806-4dca-a84a-c17f0b86c47c
[I 2024-05-18 03:32:28,518] Trial 0 finished with value: 582.6352554527333 and parameters: {'n_neighbors': 9, 'weights': 'distance', 'p': 10}. Best is trial 0 with value: 582.6352554527333.
[I 2024-05-18 03:32:28,531] Trial 1 finished with value: 640.3171944753744 and parameters: {'n_neighbors': 1, 'weights': 'uniform', 'p': 6}. Best is trial 0 with value: 582.6352554527333.
[I 2024-05-18 03:32:29,036] Trial 3 finished with value: 623.4489880408822 and parameters: {'n_neighbors': 1, 'weights': 'uniform', 'p': 1}. Best is trial 0 with value: 582.6352554527333.
[I 2024-05-18 03:32:29,094] Trial 2 finished with value: 585.0465036604953 and parameters: {'n_neighbors': 10, 'weights': 'distance', 'p': 4}. Best is trial 0 with value: 582.6352554527333.
[I 2024-05-18 03:32:29,580] Trial 4 finished with value: 578.9244116777141 and parameters: {'n_neighbors': 5, 'weights': 'distance

Lagring av resultater

In [ ]:
res_1_design = pd.DataFrame(res.dict_res_1_design)
res_1_design_opt = pd.DataFrame(res.dict_res_1_design_opt)

res_1_design.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_1_design.csv')
res_1_design_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_1_design_opt.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Velger beste modell

## Alternativ 1: Forskjellige treningsandeler

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, _, _, _ = trening_testsett_oppdeling_enzym(rå_data,
                                                       trening,
                                                       test_andel=ubenyttet_andel,
                                                       random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert

  # n_splitt
  n_splitt = n_splitt_enzym(rå_data, treningssett_markert)

  # Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
  veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_design_opt,
                               dict_res=res.dict_res_1_design,
                               treningssett_markert=treningssett_markert,
                               enzymtyper_treningssett_markert=enzymtyper_treningssett_markert,
                               testsett=testsett,
                               n_splitt=n_splitt,
                               n_gjentagelser=n_gjentagelser,
                               n_runder=n_runder_andel,
                               test_andel=test_andel,
                               trenings_andel=trenings_andel,
                               random_seed=random_seed)

  res_1_design = pd.DataFrame(res.dict_res_1_design)
  res_1_design_opt = pd.DataFrame(res.dict_res_1_design_opt)

  res_1_design.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_1_design.csv')
  res_1_design_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_1_design_opt.csv')

[I 2024-05-18 03:56:42,161] A new study created in memory with name: no-name-e89336f6-46bd-4a13-99cf-87c18ae14420
[I 2024-05-18 03:56:42,475] Trial 0 finished with value: 912.2509876448739 and parameters: {'n_neighbors': 5, 'weights': 'uniform', 'p': 1}. Best is trial 0 with value: 912.2509876448739.
[I 2024-05-18 03:56:42,656] Trial 1 finished with value: 1056.238776230448 and parameters: {'n_neighbors': 1, 'weights': 'distance', 'p': 4}. Best is trial 0 with value: 912.2509876448739.
[I 2024-05-18 03:56:42,904] Trial 2 finished with value: 1095.3844187124607 and parameters: {'n_neighbors': 8, 'weights': 'uniform', 'p': 8}. Best is trial 0 with value: 912.2509876448739.
[I 2024-05-18 03:56:43,119] Trial 3 finished with value: 977.5820314704523 and parameters: {'n_neighbors': 5, 'weights': 'uniform', 'p': 5}. Best is trial 0 with value: 912.2509876448739.
[I 2024-05-18 03:56:43,269] Trial 4 finished with value: 887.446766683126 and parameters: {'n_neighbors': 9, 'weights': 'distance', 

Lagring av resultater

In [ ]:
res_1_design = pd.DataFrame(res.dict_res_1_design)
res_1_design_opt = pd.DataFrame(res.dict_res_1_design_opt)

res_1_design.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_1_design.csv')
res_1_design_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_1_design_opt.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Alternativ 2: Hele treningssett

In [ ]:
# Definerer treningsandel
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_design,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

Enzymtyper Fordeling: Test
            Frekvens  Andel
EnzymeCode                 
E                 14  27.45
B                 13  25.49
C                 13  25.49
D                 11  21.57


In [ ]:
# Finner antall splitt etter minste antall av enzymtype
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_design_opt,
                             dict_res=res.dict_res_2_design,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder_hel,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

res_2_design = pd.DataFrame(res.dict_res_2_design)
res_2_design_opt = pd.DataFrame(res.dict_res_2_design_opt)

res_2_design.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_design.csv')
res_2_design_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_design_opt.csv')

[I 2024-05-18 05:02:28,746] A new study created in memory with name: no-name-d5a3569e-105e-4509-94ca-f7bbe1fccf11
[I 2024-05-18 05:02:37,230] Trial 0 finished with value: 626.5963702941302 and parameters: {'n_neighbors': 7, 'weights': 'distance', 'p': 2}. Best is trial 0 with value: 626.5963702941302.
[I 2024-05-18 05:02:37,458] Trial 1 finished with value: 669.6175459130309 and parameters: {'n_neighbors': 8, 'weights': 'uniform', 'p': 7}. Best is trial 0 with value: 626.5963702941302.
[I 2024-05-18 05:02:46,280] Trial 2 finished with value: 666.4169540640308 and parameters: {'n_neighbors': 5, 'weights': 'uniform', 'p': 5}. Best is trial 0 with value: 626.5963702941302.
[I 2024-05-18 05:02:46,467] Trial 3 finished with value: 647.8238931549301 and parameters: {'n_neighbors': 3, 'weights': 'uniform', 'p': 2}. Best is trial 0 with value: 626.5963702941302.
[I 2024-05-18 05:02:54,745] Trial 4 finished with value: 641.8723685134831 and parameters: {'n_neighbors': 6, 'weights': 'distance', 

Velger beste modell

## Alternativ 2: Forskjellige treningssandeler

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, _, _\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       test_ekskluder_kalkun_kylling=True)

  else:
    treningssett_markert = trening_markert

  #Enzymfordeling
  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)
  print(enzym_treningssett_markert)

  # n_splitt
  n_splitt = n_splitt_enzym(rå_data, treningssett_markert)

  veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_design_opt,
                               dict_res=res.dict_res_2_design,
                               rå_data=rå_data,
                               treningssett_markert=treningssett_markert,
                               testsett=testsett,
                               n_splitt=n_splitt,
                               n_runder=n_runder_andel,
                               test_andel=test_andel,
                               trenings_andel=trenings_andel,
                               random_seed=random_seed)

  res_2_design = pd.DataFrame(res.dict_res_2_design)
  res_2_design_opt = pd.DataFrame(res.dict_res_2_design_opt)

  res_2_design.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_design.csv')
  res_2_design_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_design_opt.csv')

Lagring av resultater

In [ ]:
res_2_design = pd.DataFrame(res.dict_res_2_design)
res_2_design_opt = pd.DataFrame(res.dict_res_2_design_opt)

res_2_design.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_design.csv')
res_2_design_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_design_opt.csv')

# NIR målinger ekskludert

In [ ]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
data_uten_NIR = data.drop(columns=NIR_kolonner)
data_design_uten_NIR = data_design.drop(columns=NIR_kolonner)

## Hele datasett

## Alternativ 1: Hele treningssett

In [ ]:
# Setter treningsandel
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_uten_NIR,
                                   test_andel=test_andel)

# Finner antall splitt etter minste antall av enzymtype
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_hel_opt_NIR,
                             dict_res=res.dict_res_1_hel_NIR,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder_hel,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

[I 2024-04-29 11:48:40,728] A new study created in memory with name: no-name-d62b36dd-29e6-415a-8349-91bdeb26d58b
[I 2024-04-29 11:48:43,126] Trial 1 finished with value: 969.4486393088273 and parameters: {'n_neighbors': 17, 'weights': 'uniform', 'p': 1}. Best is trial 1 with value: 969.4486393088273.
[I 2024-04-29 11:48:43,182] Trial 0 finished with value: 921.8369099748346 and parameters: {'n_neighbors': 8, 'weights': 'uniform', 'p': 1}. Best is trial 0 with value: 921.8369099748346.
[I 2024-04-29 11:48:44,458] Trial 2 pruned. 
[I 2024-04-29 11:48:45,247] Trial 3 finished with value: 840.2517084840687 and parameters: {'n_neighbors': 9, 'weights': 'distance', 'p': 4}. Best is trial 3 with value: 840.2517084840687.
[I 2024-04-29 11:48:46,501] Trial 4 finished with value: 842.2189330706549 and parameters: {'n_neighbors': 8, 'weights': 'distance', 'p': 8}. Best is trial 3 with value: 840.2517084840687.
[I 2024-04-29 11:48:46,594] Trial 5 pruned. 
[I 2024-04-29 11:48:47,224] Trial 6 prune

Velger beste modell

Resultater

In [ ]:
res_1_hel_NIR = pd.DataFrame(res.dict_res_1_hel_NIR)
res_1_hel_opt_NIR = pd.DataFrame(res.dict_res_1_hel_opt_NIR)

res_1_hel_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_1_hel_NIR.csv')
res_1_hel_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_1_hel_opt_NIR.csv')

## Alternativ 2: Hele treningssett

In [ ]:
# Definerer treningandel
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_uten_NIR,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

# Finner antall splitt etter minste antall av enzymtype
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_hel_opt_NIR,
                             dict_res=res.dict_res_2_hel_NIR,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder_hel,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

[I 2024-04-29 13:15:45,839] A new study created in memory with name: no-name-9c8a03ef-9006-4ab1-9b6e-eb78b5099cd5
[I 2024-04-29 13:16:01,945] Trial 0 finished with value: 853.8243444316003 and parameters: {'n_neighbors': 16, 'weights': 'distance', 'p': 2}. Best is trial 0 with value: 853.8243444316003.
[I 2024-04-29 13:16:02,005] Trial 1 finished with value: 863.6642097259589 and parameters: {'n_neighbors': 9, 'weights': 'distance', 'p': 10}. Best is trial 0 with value: 853.8243444316003.
[I 2024-04-29 13:16:18,365] Trial 3 finished with value: 858.6382790495934 and parameters: {'n_neighbors': 10, 'weights': 'uniform', 'p': 2}. Best is trial 0 with value: 853.8243444316003.
[I 2024-04-29 13:16:18,513] Trial 2 finished with value: 868.18838232223 and parameters: {'n_neighbors': 14, 'weights': 'distance', 'p': 9}. Best is trial 0 with value: 853.8243444316003.
[I 2024-04-29 13:16:36,030] Trial 4 finished with value: 886.0610195627947 and parameters: {'n_neighbors': 18, 'weights': 'unifor

Velger beste modell

Resultater

In [ ]:
res_2_hel_NIR = pd.DataFrame(res.dict_res_2_hel_NIR)
res_2_hel_opt_NIR = pd.DataFrame(res.dict_res_2_hel_opt_NIR)

res_2_hel_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_hel_NIR.csv')
res_2_hel_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_hel_opt_NIR.csv')

## Designproduksjon

## Alternativ 1: Hele treningssett

In [ ]:
# Setter treningsandel
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design_uten_NIR,
                                   test_andel=test_andel)

# Finner antall splitt etter minste antall av enzymtype
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_design_opt_NIR,
                             dict_res=res.dict_res_1_design_NIR,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder_hel,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

[I 2024-04-29 14:16:25,370] A new study created in memory with name: no-name-7b79ddd0-1c8c-47c8-9927-8ffead738ef0
[I 2024-04-29 14:16:26,599] Trial 0 finished with value: 922.6108652768614 and parameters: {'n_neighbors': 6, 'weights': 'uniform', 'p': 6}. Best is trial 0 with value: 922.6108652768614.
[I 2024-04-29 14:16:26,673] Trial 1 finished with value: 1051.3014826687368 and parameters: {'n_neighbors': 18, 'weights': 'uniform', 'p': 8}. Best is trial 0 with value: 922.6108652768614.
[I 2024-04-29 14:16:27,773] Trial 2 finished with value: 923.5343544369395 and parameters: {'n_neighbors': 8, 'weights': 'distance', 'p': 8}. Best is trial 0 with value: 922.6108652768614.
[I 2024-04-29 14:16:28,045] Trial 3 finished with value: 928.4652017684136 and parameters: {'n_neighbors': 8, 'weights': 'uniform', 'p': 2}. Best is trial 0 with value: 922.6108652768614.
[I 2024-04-29 14:16:28,636] Trial 4 pruned. 
[I 2024-04-29 14:16:29,358] Trial 5 finished with value: 889.8247444190188 and paramet

Velger beste modell

Resultater

In [ ]:
res_1_design_NIR = pd.DataFrame(res.dict_res_1_design_NIR)
res_1_design_opt_NIR = pd.DataFrame(res.dict_res_1_design_opt_NIR)

res_1_design_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_1_design_NIR.csv')
res_1_design_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_1_design_opt_NIR.csv')

## Alternativ 2: Hele treningssett

In [ ]:
# Definerer treningsandel
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_design_uten_NIR,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

# Finner antall splitt etter minste antall av enzymtype
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_design_opt_NIR,
                             dict_res=res.dict_res_2_design_NIR,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder_hel,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

[I 2024-04-29 15:43:58,237] A new study created in memory with name: no-name-9fba9bb3-3391-48a2-84a6-63105527c21b
[I 2024-04-29 15:44:11,080] Trial 0 finished with value: 877.1955077011303 and parameters: {'n_neighbors': 20, 'weights': 'distance', 'p': 6}. Best is trial 0 with value: 877.1955077011303.
[I 2024-04-29 15:44:11,292] Trial 1 finished with value: 1032.1515862395736 and parameters: {'n_neighbors': 13, 'weights': 'uniform', 'p': 9}. Best is trial 0 with value: 877.1955077011303.
[I 2024-04-29 15:44:24,290] Trial 2 finished with value: 857.9629116338382 and parameters: {'n_neighbors': 2, 'weights': 'distance', 'p': 2}. Best is trial 2 with value: 857.9629116338382.
[I 2024-04-29 15:44:24,463] Trial 3 finished with value: 1138.4973126659343 and parameters: {'n_neighbors': 20, 'weights': 'uniform', 'p': 7}. Best is trial 2 with value: 857.9629116338382.
[I 2024-04-29 15:44:37,380] Trial 4 finished with value: 859.7621890911973 and parameters: {'n_neighbors': 6, 'weights': 'unifo

Velger beste modell

Resultater

In [ ]:
res_2_design_NIR = pd.DataFrame(res.dict_res_2_design_NIR)
res_2_design_opt_NIR = pd.DataFrame(res.dict_res_2_design_opt_NIR)

res_2_design_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_design_NIR.csv')
res_2_design_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/mw/mw_resultater_2_design_opt_NIR.csv')